<a href="https://colab.research.google.com/github/debjyoti003/Important-Colab-Notebooks/blob/main/Tweets_Pre_processing_and_NER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import re, os
import seaborn as sns
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import nltk.corpus
import spacy
nltk.download("stopwords")
from nltk.corpus import stopwords
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
os.chdir('/content/drive/MyDrive/')

In [ ]:
df = pd.read_csv('extracted_tweets.csv')
df.head()

,id,conversation_id,created_at,date,time,timezone,user_id,username,name,place,tweet,language,mentions,urls,photos,replies_count,retweets_count,likes_count,hashtags,cashtags,link,retweet,quote_url,video,thumbnail,near,geo,source,user_rt_id,user_rt,retweet_id,reply_to,retweet_date,translate,trans_src,trans_dest
0,1.392400e+18,1.392400e+18,2021-05-12 14:01:43 India Standard Time,12-05-2021,14:01:43,530,8.929951e+08,ifrcafrica,IFRC Africa,NaN,"Across Africa, at any given time, Red Cross an...",en,[],['http://bit.ly/2RNeRSU'],['https://pbs.twimg.com/media/E1LJvvxWEAIRV86....,0,0,2,['floods'],[],https://twitter.com/IFRCAfrica/status/13923970...,False,NaN,1,https://pbs.twimg.com/media/E1LJvvxWEAIRV86.jpg,NaN,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
1,1.392400e+18,1.392400e+18,2021-05-12 13:55:41 India Standard Time,12-05-2021,13:55:41,530,9.085950e+17,wanjohikinguru,Wanjohi King'uru,NaN,Poor drainage in Transami means we have to end...,en,[],[],[],0,0,0,"['nairobi', 'floods']",[],https://twitter.com/WanjohiKinguru/status/1392...,False,NaN,1,https://pbs.twimg.com/ext_tw_video_thumb/13923...,NaN,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
2,1.392390e+18,1.392390e+18,2021-05-12 13:35:46 India Standard Time,12-05-2021,13:35:46,530,1.604304e+07,ncedigital,New Civil Engineer,NaN,Daily brief: 🌥️ Queen's Speech intensifies #H...,en,"[{'screen_name': 'networkrail', 'name': 'netwo...","['https://buff.ly/3eDdK1i', 'https://buff.ly/3...",['https://pbs.twimg.com/media/E1LDz05XIAAptuR....,0,2,2,"['hs2', 'engineering', 'civilengineering', 'ra...",[],https://twitter.com/ncedigital/status/13923905...,False,NaN,1,https://pbs.twimg.com/media/E1LDz05XIAAptuR.jpg,NaN,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
3,1.392390e+18,1.392390e+18,2021-05-12 13:34:05 India Standard Time,12-05-2021,13:34:05,530,2.985111e+09,malurussally,Sally Forsstrom,NaN,#climate change impacts here already with #dro...,en,[],[],[],0,0,0,"['climate', 'drought', 'bushfires', 'floods', ...",[],https://twitter.com/MalurusSally/status/139239...,False,https://twitter.com/shoebridgemlc/status/13922...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
4,1.392390e+18,1.392390e+18,2021-05-12 13:29:27 India Standard Time,12-05-2021,13:29:27,530,8.565930e+17,infonile,InfoNile,NaN,"🧵: This is Thirst in a River Basin, an interac...",en,[],['https://maps.infonile.org/covid-water-scarci...,['https://pbs.twimg.com/media/E1LCXHmXoAM1J_9....,0,0,0,"['infomap', 'rainfall', 'droughts', 'floods', ...",[],https://twitter.com/infoNILE/status/1392388939...,False,NaN,1,https://pbs.twimg.com/media/E1LCXHmXoAM1J_9.jpg,NaN,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN


In [ ]:
df.shape

(10089, 36)

In [ ]:
df.isnull().sum()

id                     0
conversation_id        0
created_at             0
date                   0
time                   0
timezone               0
user_id                0
username               0
name                   0
place              10023
tweet                  0
language               0
mentions               0
urls                   0
photos                 0
replies_count          0
retweets_count         0
likes_count            0
hashtags               0
cashtags               0
link                   0
retweet                0
quote_url           9108
video                  0
thumbnail           5942
near               10089
geo                10089
source             10089
user_rt_id         10089
user_rt            10089
retweet_id         10089
reply_to               0
retweet_date       10089
translate          10089
trans_src          10089
trans_dest         10089
dtype: int64

In [ ]:
pd.set_option('display.max_colwidth', None)

In [ ]:
# df2 = df.drop(df.columns.difference(['tweet']), axis = 1)
df2 = df[['tweet']]
df2.head()

,tweet
0,"Across Africa, at any given time, Red Cross and Red Crescent National Societies are responding to #floods or cyclones with lifesaving actions. Learn more: https://t.co/iLiQpJQbOi. https://t.co/6vhltAOkdE"
1,Poor drainage in Transami means we have to endure flooding when there is rain in Nairobi. We need to redo the drainage networks in the city. #Nairobi #Floods https://t.co/FfwBfro0KO
2,Daily brief: 🌥️ Queen's Speech intensifies #HS2 anxiety https://t.co/Oqapyv1m9s 🌥️ @networkrail station competition winner revealed https://t.co/Px3PLn9huQ 🌥️ Flood Action Plan launched https://t.co/qIWqDlDQQn #Engineering #CivilEngineering #Rail #Stations #Floods https://t.co/mQWXqpiq2C
3,#climate change impacts here already with #drought the #bushfires from August 2019 - March 2020 then #floods we need a plan for #NetZeroEmissions
4,"🧵: This is Thirst in a River Basin, an interactive #InfoMap 🌍that pinpoints the areas most vulnerable to changing #rainfall and runoff patterns - factors that contribute to #droughts and #floods - and those already facing 💧 #water scarcity in the Nile. https://t.co/gOZUu08Gxa https://t.co/uaDstL349p"


Below I have removed 'https' links and username defined by @, as links are not useful, also username won't help much in the detection of location. Whole purpose of doing this is to detect the locations from the tweets. My purpose of not removing the hastags is, as in many of the hastags we can get the location names, so I have kept it there.

In [ ]:
def cleaned_data(text):
    clean=re.sub("http\S+","",text)
    clean = clean.lower()
    clean=re.sub("@\S+","",clean)
    clean = re.sub('#', '', clean)
    # clean=re.sub("[^a-zA-Z0-9]"," ",clean)
    return clean
df2['Cleaned Tweets'] = df2['tweet'].apply(cleaned_data)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
df2.head(15)

,tweet,Cleaned Tweets
0,"Across Africa, at any given time, Red Cross and Red Crescent National Societies are responding to #floods or cyclones with lifesaving actions. Learn more: https://t.co/iLiQpJQbOi. https://t.co/6vhltAOkdE","across africa, at any given time, red cross and red crescent national societies are responding to floods or cyclones with lifesaving actions. learn more:"
1,Poor drainage in Transami means we have to endure flooding when there is rain in Nairobi. We need to redo the drainage networks in the city. #Nairobi #Floods https://t.co/FfwBfro0KO,poor drainage in transami means we have to endure flooding when there is rain in nairobi. we need to redo the drainage networks in the city. nairobi floods
2,Daily brief: 🌥️ Queen's Speech intensifies #HS2 anxiety https://t.co/Oqapyv1m9s 🌥️ @networkrail station competition winner revealed https://t.co/Px3PLn9huQ 🌥️ Flood Action Plan launched https://t.co/qIWqDlDQQn #Engineering #CivilEngineering #Rail #Stations #Floods https://t.co/mQWXqpiq2C,daily brief: 🌥️ queen's speech intensifies hs2 anxiety 🌥️ station competition winner revealed 🌥️ flood action plan launched engineering civilengineering rail stations floods
3,#climate change impacts here already with #drought the #bushfires from August 2019 - March 2020 then #floods we need a plan for #NetZeroEmissions,climate change impacts here already with drought the bushfires from august 2019 - march 2020 then floods we need a plan for netzeroemissions
4,"🧵: This is Thirst in a River Basin, an interactive #InfoMap 🌍that pinpoints the areas most vulnerable to changing #rainfall and runoff patterns - factors that contribute to #droughts and #floods - and those already facing 💧 #water scarcity in the Nile. https://t.co/gOZUu08Gxa https://t.co/uaDstL349p","🧵: this is thirst in a river basin, an interactive infomap 🌍that pinpoints the areas most vulnerable to changing rainfall and runoff patterns - factors that contribute to droughts and floods - and those already facing 💧 water scarcity in the nile."
5,ఉత్తరాఖండ్‌లో వరదలు... #Kaitvmedia #Keemedia1 #Urvasiott #Keeyoungmantra #Az7amnews #Uttarakhand #Floods #Theri https://t.co/MpgLkAMm9X,ఉత్తరాఖండ్‌లో వరదలు... kaitvmedia keemedia1 urvasiott keeyoungmantra az7amnews uttarakhand floods theri
6,"What if we could avoid Fatbergs, #Sewer Blockages, and #Floods? That's what Brian Moloney shares us in this week's #Podcast. 🎧 Fatbergs Right Ahead? Not if you Harvest the Right Data! ▶ Listen to the full episode on your favorite Podcatcher, or on https://t.co/PaCpbIkwNx https://t.co/NuVfDwoiTD","what if we could avoid fatbergs, sewer blockages, and floods? that's what brian moloney shares us in this week's podcast. 🎧 fatbergs right ahead? not if you harvest the right data! ▶ listen to the full episode on your favorite podcatcher, or on"
7,"#UPDATE 🇺🇳#UN #Somalia #Geneva #FLOODS Map by @CopernicusEMS On Tuesday, UN humanitarians said that torrential rains in parts of Somalia caused flash floods, killing at least 25 people just two weeks after the country was declared drought-stricken. https://t.co/lZH0rKlCaE","update 🇺🇳un somalia geneva floods map by on tuesday, un humanitarians said that torrential rains in parts of somalia caused flash floods, killing at least 25 people just two weeks after the country was declared drought-stricken."
8,@redcrosscanada help !!! @amnesty @HRWatchdog #FortSimpson #floods,help !!! fortsimpson floods
9,"Indian and Global #films. How have they depicted #floods and flood #victims? Welcoming Dr. Suchandra Ghatak as a speaker. TMYS Review Sept 2021 Calling for #essays, #stories, #poems on Migration, Resettlement and Displacement of Flood victims. https://t.co/yEphjmZDgt https://t.co/QIrjC6B38S","indian and global films. how have they depicted floods and flood victims? welcoming dr. suchandra ghatak as a speaker. tmys review sept 2021 calling for essays, stories, poems on migration, resettlement and displacement of flood victims."


In [ ]:
!pip install contractions

     |████████████████████████████████| 327kB 6.7MB/s 
     |████████████████████████████████| 286kB 8.8MB/s 
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.2-cp37-cp37m-linux_x86_64.whl size=85396 sha256=c15e1c31c823f25127d4c523b9d9d1c7fbbbcbc541ff861423b456059372beaf
  Stored in directory: /root/.cache/pip/wheels/3a/03/34/77e3ece0bba8b86bfac88a79f923b36d805cad63caeba38842
Successfully built pyahocorasick


In [ ]:
import contractions

**Below python package helps in expand contractions in text like don't -> do not, isn't -> is not etc..**

In [ ]:
text = '''I'll be there within 5 min. Shouldn't you be there too? 
          I'd love to see u there my dear. It's awesome to meet new friends.
          We've been waiting for this day for so long.'''
  
# creating an empty list
expanded_words = []    
for word in text.split():
  # using contractions.fix to expand the shotened words
  expanded_words.append(contractions.fix(word))   
    
expanded_text = ' '.join(expanded_words)
print('Original text: ' + text)
print('Expanded_text: ' + expanded_text)

Original text: I'll be there within 5 min. Shouldn't you be there too? 
          I'd love to see u there my dear. It's awesome to meet new friends.
          We've been waiting for this day for so long.
Expanded_text: I will be there within 5 min. should not you be there too? I would love to see you there my dear. it is awesome to meet new friends. we have been waiting for this day for so long.


In [ ]:
def expanded(text):
    expanded_words = []    
    try:
      for word in text.split():
        expanded_words.append(contractions.fix(word))
    except Exception:
      pass         
    expanded_text = ' '.join(expanded_words)
    return expanded_text

df2['Cleaned Tweets'] = df2['Cleaned Tweets'].apply(expanded)
df2.head(15)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


,tweet,Cleaned Tweets
0,"Across Africa, at any given time, Red Cross and Red Crescent National Societies are responding to #floods or cyclones with lifesaving actions. Learn more: https://t.co/iLiQpJQbOi. https://t.co/6vhltAOkdE","across africa, at any given time, red cross and red crescent national societies are responding to floods or cyclones with lifesaving actions. learn more:"
1,Poor drainage in Transami means we have to endure flooding when there is rain in Nairobi. We need to redo the drainage networks in the city. #Nairobi #Floods https://t.co/FfwBfro0KO,poor drainage in transami means we have to endure flooding when there is rain in nairobi. we need to redo the drainage networks in the city. nairobi floods
2,Daily brief: 🌥️ Queen's Speech intensifies #HS2 anxiety https://t.co/Oqapyv1m9s 🌥️ @networkrail station competition winner revealed https://t.co/Px3PLn9huQ 🌥️ Flood Action Plan launched https://t.co/qIWqDlDQQn #Engineering #CivilEngineering #Rail #Stations #Floods https://t.co/mQWXqpiq2C,daily brief: 🌥️ queen's speech intensifies hs2 anxiety 🌥️ station competition winner revealed 🌥️ flood action plan launched engineering civilengineering rail stations floods
3,#climate change impacts here already with #drought the #bushfires from August 2019 - March 2020 then #floods we need a plan for #NetZeroEmissions,climate change impacts here already with drought the bushfires from august 2019 - march 2020 then floods we need a plan for netzeroemissions
4,"🧵: This is Thirst in a River Basin, an interactive #InfoMap 🌍that pinpoints the areas most vulnerable to changing #rainfall and runoff patterns - factors that contribute to #droughts and #floods - and those already facing 💧 #water scarcity in the Nile. https://t.co/gOZUu08Gxa https://t.co/uaDstL349p","🧵: this is thirst in a river basin, an interactive infomap 🌍that pinpoints the areas most vulnerable to changing rainfall and runoff patterns - factors that contribute to droughts and floods - and those already facing 💧 water scarcity in the nile."
5,ఉత్తరాఖండ్‌లో వరదలు... #Kaitvmedia #Keemedia1 #Urvasiott #Keeyoungmantra #Az7amnews #Uttarakhand #Floods #Theri https://t.co/MpgLkAMm9X,ఉత్తరాఖండ్‌లో వరదలు... kaitvmedia keemedia1 urvasiott keeyoungmantra az7amnews uttarakhand floods theri
6,"What if we could avoid Fatbergs, #Sewer Blockages, and #Floods? That's what Brian Moloney shares us in this week's #Podcast. 🎧 Fatbergs Right Ahead? Not if you Harvest the Right Data! ▶ Listen to the full episode on your favorite Podcatcher, or on https://t.co/PaCpbIkwNx https://t.co/NuVfDwoiTD","what if we could avoid fatbergs, sewer blockages, and floods? that is what brian moloney shares us in this week's podcast. 🎧 fatbergs right ahead? not if you harvest the right data! ▶ listen to the full episode on your favorite podcatcher, or on"
7,"#UPDATE 🇺🇳#UN #Somalia #Geneva #FLOODS Map by @CopernicusEMS On Tuesday, UN humanitarians said that torrential rains in parts of Somalia caused flash floods, killing at least 25 people just two weeks after the country was declared drought-stricken. https://t.co/lZH0rKlCaE","update 🇺🇳un somalia geneva floods map by on tuesday, un humanitarians said that torrential rains in parts of somalia caused flash floods, killing at least 25 people just two weeks after the country was declared drought-stricken."
8,@redcrosscanada help !!! @amnesty @HRWatchdog #FortSimpson #floods,help !!! fortsimpson floods
9,"Indian and Global #films. How have they depicted #floods and flood #victims? Welcoming Dr. Suchandra Ghatak as a speaker. TMYS Review Sept 2021 Calling for #essays, #stories, #poems on Migration, Resettlement and Displacement of Flood victims. https://t.co/yEphjmZDgt https://t.co/QIrjC6B38S","indian and global films. how have they depicted floods and flood victims? welcoming dr. suchandra ghatak as a speaker. tmys review sept 2021 calling for essays, stories, poems on migration, resettlement and displacement of flood victims."


Below code will remove all the characters other than alphanumeric. This I could have done earlier, but is done later to make sure it doesn't remove other important things.

In [ ]:
def cleaned_further(text):
    clean=re.sub("[^a-zA-Z0-9]"," ",text)
    return clean
df2['Cleaned Tweets Further'] = df2['Cleaned Tweets'].apply(cleaned_further)
df2.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,tweet,Cleaned Tweets,Cleaned Tweets Further
0,"Across Africa, at any given time, Red Cross and Red Crescent National Societies are responding to #floods or cyclones with lifesaving actions. Learn more: https://t.co/iLiQpJQbOi. https://t.co/6vhltAOkdE","across africa, at any given time, red cross and red crescent national societies are responding to floods or cyclones with lifesaving actions. learn more:",across africa at any given time red cross and red crescent national societies are responding to floods or cyclones with lifesaving actions learn more
1,Poor drainage in Transami means we have to endure flooding when there is rain in Nairobi. We need to redo the drainage networks in the city. #Nairobi #Floods https://t.co/FfwBfro0KO,poor drainage in transami means we have to endure flooding when there is rain in nairobi. we need to redo the drainage networks in the city. nairobi floods,poor drainage in transami means we have to endure flooding when there is rain in nairobi we need to redo the drainage networks in the city nairobi floods
2,Daily brief: 🌥️ Queen's Speech intensifies #HS2 anxiety https://t.co/Oqapyv1m9s 🌥️ @networkrail station competition winner revealed https://t.co/Px3PLn9huQ 🌥️ Flood Action Plan launched https://t.co/qIWqDlDQQn #Engineering #CivilEngineering #Rail #Stations #Floods https://t.co/mQWXqpiq2C,daily brief: 🌥️ queen's speech intensifies hs2 anxiety 🌥️ station competition winner revealed 🌥️ flood action plan launched engineering civilengineering rail stations floods,daily brief queen s speech intensifies hs2 anxiety station competition winner revealed flood action plan launched engineering civilengineering rail stations floods
3,#climate change impacts here already with #drought the #bushfires from August 2019 - March 2020 then #floods we need a plan for #NetZeroEmissions,climate change impacts here already with drought the bushfires from august 2019 - march 2020 then floods we need a plan for netzeroemissions,climate change impacts here already with drought the bushfires from august 2019 march 2020 then floods we need a plan for netzeroemissions
4,"🧵: This is Thirst in a River Basin, an interactive #InfoMap 🌍that pinpoints the areas most vulnerable to changing #rainfall and runoff patterns - factors that contribute to #droughts and #floods - and those already facing 💧 #water scarcity in the Nile. https://t.co/gOZUu08Gxa https://t.co/uaDstL349p","🧵: this is thirst in a river basin, an interactive infomap 🌍that pinpoints the areas most vulnerable to changing rainfall and runoff patterns - factors that contribute to droughts and floods - and those already facing 💧 water scarcity in the nile.",this is thirst in a river basin an interactive infomap that pinpoints the areas most vulnerable to changing rainfall and runoff patterns factors that contribute to droughts and floods and those already facing water scarcity in the nile


**Almost all most of the pre-processing is done, now the only part that is remaining is extracting the locations from all the text. I will update it soon. You guys also work on it.**

In [ ]:
nlp = spacy.load('en_core_web_sm')

In [ ]:
def entity_extract(text):
  doc = nlp(text)
  return [ent.text for ent in doc.ents if ent.label_ == 'GPE']

df2['Location Name'] = df2['Cleaned Tweets Further'].apply(entity_extract)

df2.head()


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,tweet,Cleaned Tweets,Cleaned Tweets Further,Location Name
0,"Across Africa, at any given time, Red Cross and Red Crescent National Societies are responding to #floods or cyclones with lifesaving actions. Learn more: https://t.co/iLiQpJQbOi. https://t.co/6vhltAOkdE","across africa, at any given time, red cross and red crescent national societies are responding to floods or cyclones with lifesaving actions. learn more:",across africa at any given time red cross and red crescent national societies are responding to floods or cyclones with lifesaving actions learn more,[]
1,Poor drainage in Transami means we have to endure flooding when there is rain in Nairobi. We need to redo the drainage networks in the city. #Nairobi #Floods https://t.co/FfwBfro0KO,poor drainage in transami means we have to endure flooding when there is rain in nairobi. we need to redo the drainage networks in the city. nairobi floods,poor drainage in transami means we have to endure flooding when there is rain in nairobi we need to redo the drainage networks in the city nairobi floods,"[transami, nairobi, nairobi]"
2,Daily brief: 🌥️ Queen's Speech intensifies #HS2 anxiety https://t.co/Oqapyv1m9s 🌥️ @networkrail station competition winner revealed https://t.co/Px3PLn9huQ 🌥️ Flood Action Plan launched https://t.co/qIWqDlDQQn #Engineering #CivilEngineering #Rail #Stations #Floods https://t.co/mQWXqpiq2C,daily brief: 🌥️ queen's speech intensifies hs2 anxiety 🌥️ station competition winner revealed 🌥️ flood action plan launched engineering civilengineering rail stations floods,daily brief queen s speech intensifies hs2 anxiety station competition winner revealed flood action plan launched engineering civilengineering rail stations floods,[]
3,#climate change impacts here already with #drought the #bushfires from August 2019 - March 2020 then #floods we need a plan for #NetZeroEmissions,climate change impacts here already with drought the bushfires from august 2019 - march 2020 then floods we need a plan for netzeroemissions,climate change impacts here already with drought the bushfires from august 2019 march 2020 then floods we need a plan for netzeroemissions,[]
4,"🧵: This is Thirst in a River Basin, an interactive #InfoMap 🌍that pinpoints the areas most vulnerable to changing #rainfall and runoff patterns - factors that contribute to #droughts and #floods - and those already facing 💧 #water scarcity in the Nile. https://t.co/gOZUu08Gxa https://t.co/uaDstL349p","🧵: this is thirst in a river basin, an interactive infomap 🌍that pinpoints the areas most vulnerable to changing rainfall and runoff patterns - factors that contribute to droughts and floods - and those already facing 💧 water scarcity in the nile.",this is thirst in a river basin an interactive infomap that pinpoints the areas most vulnerable to changing rainfall and runoff patterns factors that contribute to droughts and floods and those already facing water scarcity in the nile,[infomap]


In [ ]:
df2.head(50)

,tweet,Cleaned Tweets,Cleaned Tweets Further,Location Name
0,"Across Africa, at any given time, Red Cross and Red Crescent National Societies are responding to #floods or cyclones with lifesaving actions. Learn more: https://t.co/iLiQpJQbOi. https://t.co/6vhltAOkdE","across africa, at any given time, red cross and red crescent national societies are responding to floods or cyclones with lifesaving actions. learn more:",across africa at any given time red cross and red crescent national societies are responding to floods or cyclones with lifesaving actions learn more,[]
1,Poor drainage in Transami means we have to endure flooding when there is rain in Nairobi. We need to redo the drainage networks in the city. #Nairobi #Floods https://t.co/FfwBfro0KO,poor drainage in transami means we have to endure flooding when there is rain in nairobi. we need to redo the drainage networks in the city. nairobi floods,poor drainage in transami means we have to endure flooding when there is rain in nairobi we need to redo the drainage networks in the city nairobi floods,"[transami, nairobi, nairobi]"
2,Daily brief: 🌥️ Queen's Speech intensifies #HS2 anxiety https://t.co/Oqapyv1m9s 🌥️ @networkrail station competition winner revealed https://t.co/Px3PLn9huQ 🌥️ Flood Action Plan launched https://t.co/qIWqDlDQQn #Engineering #CivilEngineering #Rail #Stations #Floods https://t.co/mQWXqpiq2C,daily brief: 🌥️ queen's speech intensifies hs2 anxiety 🌥️ station competition winner revealed 🌥️ flood action plan launched engineering civilengineering rail stations floods,daily brief queen s speech intensifies hs2 anxiety station competition winner revealed flood action plan launched engineering civilengineering rail stations floods,[]
3,#climate change impacts here already with #drought the #bushfires from August 2019 - March 2020 then #floods we need a plan for #NetZeroEmissions,climate change impacts here already with drought the bushfires from august 2019 - march 2020 then floods we need a plan for netzeroemissions,climate change impacts here already with drought the bushfires from august 2019 march 2020 then floods we need a plan for netzeroemissions,[]
4,"🧵: This is Thirst in a River Basin, an interactive #InfoMap 🌍that pinpoints the areas most vulnerable to changing #rainfall and runoff patterns - factors that contribute to #droughts and #floods - and those already facing 💧 #water scarcity in the Nile. https://t.co/gOZUu08Gxa https://t.co/uaDstL349p","🧵: this is thirst in a river basin, an interactive infomap 🌍that pinpoints the areas most vulnerable to changing rainfall and runoff patterns - factors that contribute to droughts and floods - and those already facing 💧 water scarcity in the nile.",this is thirst in a river basin an interactive infomap that pinpoints the areas most vulnerable to changing rainfall and runoff patterns factors that contribute to droughts and floods and those already facing water scarcity in the nile,[infomap]
5,ఉత్తరాఖండ్‌లో వరదలు... #Kaitvmedia #Keemedia1 #Urvasiott #Keeyoungmantra #Az7amnews #Uttarakhand #Floods #Theri https://t.co/MpgLkAMm9X,ఉత్తరాఖండ్‌లో వరదలు... kaitvmedia keemedia1 urvasiott keeyoungmantra az7amnews uttarakhand floods theri,kaitvmedia keemedia1 urvasiott keeyoungmantra az7amnews uttarakhand floods theri,[]
6,"What if we could avoid Fatbergs, #Sewer Blockages, and #Floods? That's what Brian Moloney shares us in this week's #Podcast. 🎧 Fatbergs Right Ahead? Not if you Harvest the Right Data! ▶ Listen to the full episode on your favorite Podcatcher, or on https://t.co/PaCpbIkwNx https://t.co/NuVfDwoiTD","what if we could avoid fatbergs, sewer blockages, and floods? that is what brian moloney shares us in this week's podcast. 🎧 fatbergs right ahead? not if you harvest the right data! ▶ listen to the full episode on your favorite podcatcher, or on",what if we could avoid fatbergs sewer blockages and floods that is what brian moloney shares us in this week s podcast fatbergs right ahead not if you harvest th